 Import all necessary librarys

In [1]:
import numpy as np
import sys, os
import time
import matplotlib.pyplot as plt
import sqlite3 as sql

%matplotlib qt
plt.ion()

savedata = r'd:\Documenten\Documents\GitHub\Active-Vibration-Stabilization\Meetfiles\data\2023-04-23'
datetime = '2023-04-23 - 190807'

os.chdir(savedata)
print(os.getcwd())

f_min = 0.50
f_max = 3.00

d:\Documenten\Documents\GitHub\Active-Vibration-Stabilization\Meetfiles\data\2023-04-23


In [2]:
DISTANCES = {
'gnx':1.3750, 'gex':0.0000, 'gtx':0.0559,
'gny':0.0000, 'gey':1.3750, 'gty':0.3449,
'gnz':0.7528, 'gez':0.7528, 'gtz':3.2772,
}

AMAT = np.matrix([
#    x | y | z |        rx         |        ry         |          rz
    [0 , 1 , 0 , -DISTANCES['gnz'] ,                 0 , -DISTANCES['gnx']],    # northY
    [0 , 0 , 1 , -DISTANCES['gny'] ,  DISTANCES['gnx'] ,                 0],    # northZ
    [1 , 0 , 0 ,                 0 ,  DISTANCES['gez'] ,  DISTANCES['gey']],    # eastX
    [0 , 0 , 1 , -DISTANCES['gey'] ,  DISTANCES['gex'] ,                 0],    # eastZ
    [1 , 0 , 0 ,                 0 , -DISTANCES['gtz'] ,  DISTANCES['gty']],    # topX
    [0 , 1 , 0 ,  DISTANCES['gtz'] ,                 0 , -DISTANCES['gtx']],    # topY
])
AMAT_INV = np.linalg.inv(AMAT)

Loop through all Database files and timstamp files.</br>
and calculate the x,y,z,rx,ry,rz components</br>
Calculate the fourier transform of the z component</br>


```
data = [
    [t1, t2, t3, ..., tn],   # Array of time data
    [s1, s2, s3, ..., sn],   # Array of north Y
    [s1, s2, s3, ..., sn],   # Array of north Z
    [s1, s2, s3, ..., sn],   # Array of East  X
    [s1, s2, s3, ..., sn],   # Array of East  Z
    [s1, s2, s3, ..., sn],   # Array of Top   X
    [s1, s2, s3, ..., sn]    # Array of Top   Y
]
```
In this structure, the first array contains time data for each reading, while the other 6 arrays contain the sensor readings. The `n` represents the number of readings that are stored in the array. Each row of data represents a single reading, with the time and sensor readings for that particular reading stored in the corresponding columns.

To access a specific value in the data array, you would use the syntax `data[row_index,column_index]`. For example, to access the time value for the third reading, you would use `data[0,2]`, since the time data is stored in the first array and the third column. Similarly, to access the value of the East Z sensor reading for the fifth reading, you would use `data[4,5]`, since the fourth sensor reading is stored in the fifth row and fourth column of the array.


```
out = [
    [ x1,  x2,  x3, ...,  xn],   # Array of linear x movement
    [ y1,  y2,  y3, ...,  yn],   # Array of linear y movement
    [ z1,  z2,  z3, ...,  zn],   # Array of linear z movement
    [rx1, rx2, rx3, ..., rxn],   # Array of rotation around x
    [ry1, ry2, ry3, ..., ryn],   # Array of rotation around y
    [rz1, rz2, rz3, ..., rzn],   # Array of rotation around z
]
```

The 6xn NumPy array contains transformed values along the 6 degrees of freedom. The six degrees of freedom represent the ways in which an object can move or change orientation in three-dimensional space. These degrees of freedom include translation along the x, y, and z axes, as well as rotation around these same axes.

The array is structured such that each row represents a specific transformation applied to the object. The first column contains the translation along the x-axis, the second column contains the translation along the y-axis, and the third column contains the translation along the z-axis. The fourth, fifth, and sixth columns contain the rotation around the x, y, and z axes, respectively.



In [3]:
# try:
#     os.mkdir('pictures')
# except:
#     print('Directory already exists')
transferFunction = np.empty((0,6,4),np.complex_)

# fig1, fig1_axes = plt.subplots(1,1,sharex=True,gridspec_kw=dict(hspace=0))
# fig1_manager = plt.get_current_fig_manager()
# fig1_manager.window.showMaximized()

# fig1_axes.set_xscale('log')
# fig1_axes.set_yscale('linear')
# fig1_axes.set_xlim(1e-3,1e2)
# fig1_axes.set_ylim(-0.05, 1.5)
# fig1_axes.set_xlabel('Frequencies $[Hz]\longrightarrow$')
# fig1_axes.set_ylabel(r'Amplitude $[\frac{mm}{s}]\longrightarrow$')

# plt.pause(0.002)
# fig1.tight_layout()
# plt.pause(0.002)

# fig1_fourier, = fig1_axes.plot([], [], 'k')
# fig1_max_point, = fig1_axes.plot([],[],'ro')


for f in np.arange(f_min,f_max+0.05, 0.05):
    # fig1_manager.set_window_title(f'{f:.2f}')
    file = f'{datetime} ({f:.2f})'
    db = sql.connect(f'{file}.db')
    cur = db.cursor()

    timestamps = np.load(f'{file}.npy')

    print(f'{f:.2f}')
    timestamp = np.where(np.round(timestamps,2) == np.round(f,2))[0][0]
    t_min = timestamps[timestamp+1]
    t_max = timestamps[timestamp+2]

    print(f'f\t=\t{timestamps[timestamp]}\nt_min\t=\t{t_min}\nt_max\t=\t{ t_max}')

    meas0 = np.transpose(cur.execute(f'SELECT t,northY,northZ,eastX,eastZ,topX,topY FROM guralp WHERE t BETWEEN {timestamps[1]} AND {timestamps[2]} ').fetchall()) # t,northY,northZ,eastX,eastZ,topX,topY
    meas0 = np.mat(meas0[1:])
    meas0 = np.array(AMAT_INV*meas0)

    data = np.transpose(cur.execute(f'SELECT t,northY,northZ,eastX,eastZ,topX,topY FROM guralp WHERE t BETWEEN {t_min} AND {t_max} ').fetchall())
    meas = np.mat(data[1:])
    out = np.array(AMAT_INV*meas)

    dt = np.average([h-l for h,l in zip(data[0][1:],data[0])])
    print((data[0,-1]-data[0,0])*f)
    temp_transferFunction = np.empty((0,4),np.complex_)
    for axis, meas0_axis,out_axis  in zip('x y z rx ry rz'.split(), meas0, out):
        fourier_transform0 = 2 * np.fft.fft(meas0_axis, len(meas0_axis))[range(int(len(out_axis)/2))] / len(meas0_axis)
        fourier_transform = 2 * np.fft.fft(out_axis, len(out_axis))[range(int(len(out_axis)/2))] / len(out_axis)
        fourier_frequencies = np.fft.fftfreq(len(out_axis),dt)[range(int(len(out_axis)/2))]

        Hz = np.argmin(np.abs((f * np.ones_like(fourier_frequencies)) - fourier_frequencies))  # `Hz` is the index of `f` in the array fourier_frequencies
        temp_transferFunction = np.append(temp_transferFunction, [[f, fourier_transform0[Hz], fourier_transform[Hz], fourier_transform0[Hz]-fourier_transform[Hz]]], 0)

        max_amp = np.max(np.abs(fourier_transform[(0.1 < fourier_frequencies) * (fourier_frequencies < 10)]))
        max_Hz = np.where(np.abs(fourier_transform) == max_amp)[0][0]
        freq = fourier_frequencies[max_Hz]
        amp = np.abs(fourier_transform[max_Hz])
        phase = np.rad2deg(np.angle(fourier_transform[max_Hz]))
    
        label = fr'''f={freq:.4f}[Hz] A = {amp:.2f} $\varphi$ = {phase:.2f} [rad]'''
        print(f'{axis:<2}: {label}')

        # fig1_fourier.set_data(fourier_frequencies, np.abs(fourier_transform))
        # fig1_axes.set_title(f'Fourier transform of the signal along the {axis}-axis', dict(fontsize=48))

        # fig1_anno = fig1_axes.annotate(label,(freq,amp), textcoords="offset points", xytext=(30,10), ha='center')
        # fig1_max_point.set_data([freq],[amp])

        # plt.pause(0.002)
        # fig1.tight_layout()
        # plt.pause(0.002)

        # title = f'Fourier Transform ({fig1_manager.get_window_title()} Hz, 2 V, {axis})'
        # fig1.savefig(f'pictures/{title}.png', transparent=True,dpi=600,format='png')
        # fig1_anno.remove()
        
    db.close()

    transferFunction = np.append(transferFunction, [temp_transferFunction], 0)
# plt.close(fig1)
np.save(f'Full Transfer Function',transferFunction, True)

0.50
f	=	0.5
t_min	=	1020.00341796875
t_max	=	2020.004150390625
500.0
x : f=1.7500[Hz] A = 1.36 $\varphi$ = -13.45 [rad]
y : f=1.7500[Hz] A = 0.95 $\varphi$ = 142.15 [rad]
z : f=2.0000[Hz] A = 0.41 $\varphi$ = 1.07 [rad]
rx: f=2.0000[Hz] A = 0.26 $\varphi$ = -6.23 [rad]
ry: f=1.7500[Hz] A = 0.24 $\varphi$ = -27.65 [rad]
rz: f=1.7500[Hz] A = 0.91 $\varphi$ = 151.17 [rad]
0.55
f	=	0.55
t_min	=	929.238037109375
t_max	=	1838.329345703125
499.99995117187507
x : f=1.6500[Hz] A = 1.17 $\varphi$ = 156.30 [rad]
y : f=1.6500[Hz] A = 0.72 $\varphi$ = -49.64 [rad]
z : f=2.2000[Hz] A = 0.15 $\varphi$ = 49.06 [rad]
rx: f=1.6500[Hz] A = 0.14 $\varphi$ = -176.38 [rad]
ry: f=1.6500[Hz] A = 0.20 $\varphi$ = 128.80 [rad]
rz: f=1.6500[Hz] A = 0.63 $\varphi$ = -27.78 [rad]
0.60
f	=	0.6
t_min	=	853.4912109375
t_max	=	1686.8251953125
499.9998046875001
x : f=1.8000[Hz] A = 1.43 $\varphi$ = 86.53 [rad]
y : f=1.8000[Hz] A = 1.12 $\varphi$ = -108.92 [rad]
z : f=1.2000[Hz] A = 0.35 $\varphi$ = -150.04 [rad]
rx: f

In [4]:
# print(fourier_frequencies[[np.argmin(np.abs((i*np.ones_like(fourier_frequencies))-fourier_frequencies)) for i in np.arange(0.5,3.05,.05)]])
# print(transferFunction)
# tf = np.array([
#     fourier_frequencies[[np.argmin(np.abs((i*np.ones_like(fourier_frequencies))-fourier_frequencies)) for i in np.arange(0.5,3.05,.05)]],
#     fourier_transform[[np.argmin(np.abs((i*np.ones_like(fourier_frequencies))-fourier_frequencies)) for i in np.arange(0.5,3.05,.05)]],
#     transferFunction,
#     transferFunction-fourier_transform[[np.argmin(np.abs((i*np.ones_like(fourier_frequencies))-fourier_frequencies)) for i in np.arange(0.5,3.05,.05)]]
# ])
# tf = np.transpose(tf)







In [5]:
# transferFunction = np.load(f'Full Transfer Function (raw data).npy')
# transferFunction = np.delete(transferFunction, [2,5,12,14,15,17,24,25,29,41,42,43,44, 46,47,49],0)

# fig2, (bode_amplitude, bode_phase) = plt.subplots(2,1)
# fig2_manager = plt.get_current_fig_manager()
# fig2_manager.set_window_title('bodeplot West Z')
# fig2_manager.window.showMaximized()

# bode_amplitude.sharex(bode_phase)

# bode_amplitude.set_title('West Z actuator Bode plot')
# bode_amplitude.set_xscale('log'); bode_amplitude.set_yscale('log')
# bode_amplitude.set_ylabel('Magnitude $|H(s)|$')
# bode_amplitude.set_xlabel('Frequency $[Hz]$')
# bode_amplitude.grid(True,'both')
# bode_amplitude.set_xlim(0.5*0.95,3.0*1.05)

# bode_phase.set_ylabel('fase $arg(H(s))$  $[deg]$')
# bode_phase.set_xlabel('Frequency $[Hz]$')
# bode_phase.grid(True, 'both')
# bode_phase.set_ylim(-450,200)

# frequencies = np.abs(transferFunction[:,:,0])
# frequencies
# amplitudes = np.abs(transferFunction[:,:,3])
# bode_amplitude.plot(frequencies,amplitudes, 'o-',label= 'topX topY topZ northX northY northZ eastX eastY eastZ'.split())

# phases = np.rad2deg(np.angle(transferFunction[:,:,3]))
# for i, ph in enumerate(phases):
#     for j, p in enumerate(ph):
#         phases[i,j] = p if p <= 0 else p -360
# # phases[0:4,[0]] += 360
# # phases[17:23,[1]] -= 360
# # phases[0:3,[2]] += 360
# # phases[[0],[5]] += 360
# # phases[15:23,[5]] -= 360


# bode_phase.plot(frequencies, phases, 'o-', label='topX topY topZ northX northY northZ eastX eastY eastZ'.split())

# bode_amplitude.legend()
# bode_phase.legend()

# plt.pause(0.002)
# fig2.tight_layout()

In [6]:
# for i,trans in enumerate(transferFunction):
#     bode_amplitude.annotate(f'{i}',(trans[0],np.abs(trans[3])), textcoords="offset points", xytext=(30,10), ha='center')

# max_amp = np.max(amplitudes)
# max_amp_i = np.argmax(amplitudes)
# bode_amplitude.scatter(transferFunction[max_amp_i,0], max_amp, color='#000000')



# _3db = 10**(-3/20)*max_amp * np.ones_like(amplitudes)

# _3db_l = np.argmin(np.abs(_3db-amplitudes)[max_amp_i:])
# _3db_h = np.argmin(np.abs(_3db-amplitudes)[:max_amp_i])

# bode_amplitude.plot([transferFunction[_3db_l,0],transferFunction[_3db_h,0]], [10**(-3/20)*max_amp, 10**(-3/20)*max_amp] )

# print(_3db,'\n', _3db_l,'\n', _3db_h,'\n', max_amp_i)

In [7]:
# fig3, nyquist = plt.subplots(1,1)
# fig3.set_figheight(12)
# fig3.set_figwidth(12)
# nyquist.grid(True,'major')
# nyquist.legend()
# nyquist.set_aspect('equal')

# nyquist.set_xlabel('Real')
# nyquist.set_ylabel('Imaginary')
# nyquist.axvline(0,color='k')
# nyquist.axhline(0,color='k')
# plt.pause(0.002)
# fig3.tight_layout()
# plt.show()
# n, = nyquist.plot([],[],label='')
# nyquist.set_xlim(-2,2)
# nyquist.set_ylim(-2,2)
# for i,ax in enumerate(['x','y','z','rx', 'ry','rz']):
#     x = amplitudes[:,i]*np.cos(phases[:,i])
#     y = amplitudes[:,i]*np.sin(phases[:,i])
#     n.set_data(x,y)
#     n.set_label(ax)
#     nyquist.legend()
#     plt.pause(0.002)
#     plt.savefig(f'pictures/Nyquist [{ax}].png')

